In [2]:
from numpy import asarray
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import exp
import torch
# calculate the inception score for p(y|x)
def calculate_inception_score(p_yx, eps=1E-16):
    # calculate p(y)
    p_y = expand_dims(p_yx.mean(axis=0), 0)
    # kl divergence for each image
    kl_d = p_yx * (log(p_yx + eps)- log(p_y + eps))
    # sum over classes
    sum_kl_d = kl_d.sum(axis=1)
    # average over images
    avg_kl_d = mean(sum_kl_d)
    # undo the logs
    is_score = exp(avg_kl_d)
    return is_score
 # conditional probabilities for low quality images
p_yx = asarray([[1, 0, 0], [0.1, 0.4, 0.3], [0.4, 0, 0]])
score = calculate_inception_score(p_yx)
print(score)

1.4977912510557916


In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [50]:
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
images = [Image.open('dog.jpg'), Image.open('cat.jpg')]
images_list = torch.vstack([preprocess(image).unsqueeze(0) for image in images])

torch.Size([2, 3, 299, 299])

In [127]:
from torchvision import transforms
import numpy as np
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
def calculate_inception_score(images, n_split=10, eps=1E-16):
    model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True).to(device)
    model.eval()
    preprocess = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    images_list = torch.vstack([preprocess(image).unsqueeze(0) for image in images]).to(device)
    n_part = np.floor(images_list.shape[0] / n_split)
    scores = list()
    for i in range(n_split):
        ix_start, ix_end = i * n_part, (i+1) * n_part
        subset = images_list
        p_yx = torch.nn.functional.softmax(model(subset), dim = 1)
        p_y = expand_dims(p_yx.detach().cpu().mean(axis=0), 0)
        p_yx = p_yx.detach().cpu().numpy()
        kl_d = p_yx * (log(p_yx + eps)- log(p_y + eps))
        sum_kl_d = kl_d.sum(axis=1)
        avg_kl_d = mean(sum_kl_d)
        is_score = exp(avg_kl_d)
        scores.append(is_score)
    is_avg, is_std = mean(scores), np.std(scores)
    return is_avg, is_std

In [128]:
calculate_inception_score([Image.open('dog.jpg'), Image.open('cat.jpg'), Image.open('perro.jpg')])

Using cache found in C:\Users\USER/.cache\torch\hub\pytorch_vision_v0.10.0


(2.6954465, 2.3841858e-07)